# Validity Checks
This section tests the results in various ways to find errors
## Visual validity check
### Sum of capacity by fuel type

In [ ]:
import urllib.request
import csv
import pandas as pd
import numpy as np
import posixpath
import urllib.parse
import datetime  
import re
import os.path
import yaml  # http://pyyaml.org/, pip install pyyaml, conda install pyyaml
import json
import subprocess
import sqlite3

from bokeh.charts import Scatter, show
from bokeh.io import output_notebook
output_notebook()
%matplotlib inline
import logging
logger = logging.getLogger('notebook')
logger.setLevel('INFO')
nb_root_logger = logging.getLogger()
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                              datefmt='%d %b %Y %H:%M:%S')
nb_root_logger.handlers[0].setFormatter(formatter)

#create download and output folders if they do not exist
if not os.path.exists('downloads/'): os.makedirs('downloads/')
if not os.path.exists('output/'): os.makedirs('output/')
if not os.path.exists('output/datapackage_powerplants_germany'): os.makedirs('output/datapackage_powerplants_germany')    
if not os.path.exists('output/datapackage_powerplants_germany/original_data/'): os.makedirs('output/datapackage_powerplants_germany/original_data/')

# Specify the source path:

In [ ]:
source_path = 'output/datapackage_powerplants_germany/power_plants_germany.csv'

# Define functions

This section defines functions used multiple times within this script

# Downloads

## Download the BNetzA power plant list

This section downloads the BNetzA power plant list and converts it to a pandas data frame

In [ ]:
plantlist=pd.read_csv(source_path, 
               #skiprows=0,
               #sep=';',  # CSV field separator, default is ','
               #thousands='.',  # Thousands separator, default is ','
               #decimal=',',  # Decimal separator, default is '.')  
               encoding='utf-8')
plantlist.head()
plantlist = plantlist.set_index('Unnamed: 0')
plantlist = plantlist.reindex(plantlist.index.rename('id'))
plantlist.head()

### Capacities by plant status

In [ ]:
pivot_status_capacity = pd.pivot_table(
                        plantlist, 
                        values='capacity',
                        columns='status',
                        index='fuel', 
                        aggfunc=np.sum
                        )
pivot_status_capacity.sort_values(by='operating', inplace=True, ascending=0)
#pivot_status_capacity
pivot_status_capacity_plot=pivot_status_capacity.plot(kind='barh', stacked=True,legend=True, figsize=(12, 6))
pivot_status_capacity_plot.set_xlabel("MW")
pivot_status_capacity_plot

### Power plant age

In [ ]:
plantlist_filtered = plantlist#[plantlist.fuel=='coal']
pivot_age_capacity = pd.pivot_table(
                        plantlist_filtered, 
                        values='capacity',
                        columns='fuel',
                        index='commissioned', 
#                        index='fuel',
#                        columns='commissioned',     
                        aggfunc=np.sum,
                        dropna=True
                        )
#pivot_age_capacity
pivot_age_capacity_plot=pivot_age_capacity.plot(kind='bar', stacked=True,legend=True, figsize=(17, 10))
pivot_age_capacity_plot.set_ylabel("MW")
pivot_age_capacity_plot

### Block size vs year of commissioning
This chart is suitable to check outliers of commissioning years and block sizes. 
In theory, there should be no unexpected values, e.g. all commissioning years should be greater than 1900. 
Block sizes above 2000 MW are also unlikely.

In [ ]:
plantlist_for_plot = plantlist.copy(deep=True)
plantlist_for_plot['capacity_float'] = pd.to_numeric(plantlist_for_plot['capacity'], errors='coerce')
plantlist_for_plot['commissioned_float'] = pd.to_numeric(plantlist_for_plot['commissioned'], errors='coerce')
plot_blocksize_year = Scatter(plantlist_for_plot, 
                              notebook=True, 
                              x='commissioned_float', 
                              y='capacity_float',
                              color='fuel', 
                              title='Block-Size vs Year of Commissioning', 
                              xlabel='Year', 
                              ylabel='MW',
                              legend="top_left",
                              height=500,
                              width=700,
                             )
show(plot_blocksize_year)

## Logical checks
### Every power plant needs a capacity
List all entries with zero capacity.

In [ ]:
plantlist[plantlist.capacity == 0]

### Commissioning Dates

In [ ]:
#Show all Plants with commisioning dates below 1900 
plantlist[plantlist['commissioned'] <=1900]

In [ ]:
#Show all Plants with invalid commisioning dates
plantlist[plantlist['commissioned'].isnull()]

### Compare UBA and BNetzA data
#### Postcodes of BNetzA and UBA lists should match

In [ ]:
# List all entries with diverging postcodes (if a postcode is given)
#plantlist[(plantlist['uba_postcode'].notnull() == True) & (pd.to_numeric(plantlist.postcode, errors='coerce') != pd.to_numeric(plantlist.uba_postcode, errors='coerce'))]

#### Compare Installed capacities

In [ ]:
capacitycomparison = pd.DataFrame(plantlist.capacity / plantlist.capacity_uba)
capacitycomparison['Name'] = plantlist.name
capacitycomparison['Block'] = plantlist.block
capacitycomparison['BnetzaCapacity'] = plantlist.capacity
capacitycomparison['UBACapacity'] = plantlist.capacity_uba
capacitycomparison.dropna(inplace=True)
capacitycomparison.sort_values(by=0)

#### Compare Comissioning Years

In [ ]:
#commissioningcomparison = pd.DataFrame(plantlist.commissioned)
#commissioningcomparison['UBACommissioned'] = plantlist.uba_commissioned
#commissioningcomparison['commissioned_float'] = pd.to_numeric(commissioningcomparison['commissioned'],errors='coerce')
#commissioningcomparison['UBACommissioned_float'] = pd.to_numeric(commissioningcomparison['UBACommissioned'],errors='coerce')

#commissioningcomparison['Faktor'] = commissioningcomparison.commissioned_float - commissioningcomparison.UBACommissioned_float
#commissioningcomparison['Name'] = plantlist.name
#commissioningcomparison['Block'] = plantlist.block

#commissioningcomparison.dropna(subset = ['commissioned'], inplace=True)
#commissioningcomparison.dropna(subset = ['UBACommissioned'], inplace=True)
#commissioningcomparison = commissioningcomparison.sort_values(by='Faktor')

#commissioningcomparison